In [1]:
import importlib
import pandas as pd
import xarray as xr
import numpy as np
import sys
import math
import xesmf as xe
from scipy import signal

from ecpaper_utils import readdata_utils as read
from ecpaper_utils import calendar_utils as cal
from ecpaper_utils import averaging_utils as avg
from ecpaper_utils import shapefile_utils as shp

importlib.reload(read)
importlib.reload(cal)
importlib.reload(avg)
importlib.reload(shp)

import warnings
warnings.filterwarnings('ignore')

Set up model names, dates and CLIVAR_LE location.  Using 1950-2016 instead of 1948-2014 since CanESM2 starts in 1950.

In [2]:
modnames=['CanESM2','CSIRO-Mk3-6-0','GFDL-CM3','MPI-ESM','CESM1-CAM5']
modnames_small=['canesm2','csiro_mk36','gfdl_cm3','mpi','cesm']
#modnames=['CanESM2','CSIRO-Mk3-6-0','CESM1-CAM5','GFDL-CM3','MPI-ESM']
#modnames_small=['canesm2','csiro_mk36','cesm','gfdl_cm3','mpi']
basedir="/glade/collections/cdg/data/CLIVAR_LE/"
ybegp=1950 ; monbegp=1 ; yendp=2016 ; monendp=12
ybegf=2006 ; monbegf=1 ; yendf=2099 ; monendf=12
datebegp=str(ybegp)+"-"+str(monbegp).zfill(2)
dateendp=str(yendp)+"-"+str(monendp).zfill(2)
datebegf=str(ybegf)+"-"+str(monbegf).zfill(2)
dateendf=str(yendf)+"-"+str(monendf).zfill(2)

nmonthsp=(yendp-ybegp-1)*12 + (12-monbegp+1) + monendp
nmonthsf=(yendf-ybegf-1)*12 + (12-monbegf+1) + monendf

pathout="/glade/scratch/islas/python/ecpaper2020/DATASORT/CALP/DATA/"
shpfile="/glade/scratch/islas/python/ecpaper2020/tmp/shpfiles/USA/gadm36_USA_1.shp"

Loop over models and members and output the DJF correlation between detrended Nino3.4 and California precipitation

In [4]:
grid_out = xr.Dataset({'lat': (['lat'], np.arange(-90,91,1.0)), 'lon': (['lon'], np.arange(0,360,1.0))})
nlon = grid_out["lon"].size ; nlat=grid_out["lat"].size

for imod in range(len(modnames)):
    print(modnames[imod])
    moddirpr=basedir+modnames_small[imod]+"_lens/Amon/pr/"
    moddirts=basedir+modnames_small[imod]+"_lens/Amon/ts/"
    memfiles = !cd $moddirpr && ls *rcp85*.nc
    nmems=len(memfiles)
    reusewgt=False
    wgtfile="/glade/scratch/islas/python/tmp/wgtfile.nc"
    
    for imem in range(len(memfiles)):
        fnamepr=memfiles[imem]
        fnamesplit=fnamepr.split("_")
        fnamesplit=fnamesplit[1:len(fnamesplit)+1]
        fnamets="_".join(fnamesplit)
        fnamets="ts_"+fnamets
        print(fnamepr)
        print(fnamets)
        prp = read.read_sfc(moddirpr+fnamepr, datebegp, dateendp)
        tsp = read.read_sfc(moddirts+fnamets, datebegp, dateendp)
        prf = read.read_sfc(moddirpr+fnamepr, datebegf, dateendf)
        tsf = read.read_sfc(moddirts+fnamets, datebegf, dateendf)
        
        # check sizes
        if (prp.time.size != nmonthsp):
            print("somethings's wrong, "+str(nmonthsp)+
                  "expected, but got "+str(prp.time.size))
            sys.exit("failure at past for "+fnamepr)
            
        if (tsp.time.size != nmonthsp):
            print("something's wrong, "+str(nmonths)+
                 "expected, but got "+str(tsp.time.size))
            sys.exit("failure at past for "+fnamets)
        
        if (prf.time.size != nmonthsf):
            print("somethings's wrong, "+str(nmonthsf)+
                  "expected, but got "+str(prf.time.size))
            sys.exit("failure at future for "+fnamepr)
            
        if (tsf.time.size != nmonthsf):
            print("something's wrong, "+str(nmonths)+
                 "expected, but got "+str(tsf.time.size))
            sys.exit("failure at past for "+fnamets)    
            
        prdjfp=cal.season_ts(prp,"pr",season="DJF")
        tsdjfp=cal.season_ts(tsp,"ts",season="DJF")
        prdjff=cal.season_ts(prf,"pr",season="DJF")
        tsdjff=cal.season_ts(tsf,"ts",season="DJF")
 
        if (imem == 0):
            regridder = xe.Regridder(prdjfp, grid_out, 'bilinear',
                                periodic=True, reuse_weights=reusewgt,
                                filename=wgtfile)
    
        prdjfpinterp = regridder(prdjfp)
        prdjffinterp = regridder(prdjff)
        tsdjfpinterp = regridder(tsdjfp)
        tsdjffinterp = regridder(tsdjff)

        nino34p = avg.cosweightlonlat(tsdjfpinterp, 190, 240, -5, 5)
        nino34f = avg.cosweightlonlat(tsdjffinterp, 190, 240, -5, 5)
            
        if (imem == 0):
            mask = shp.maskgen(shpfile, prdjfpinterp, ['California'])
            
            nino34_past=xr.DataArray(np.zeros([nmems, nino34p.time.size]),
                                    coords=[np.arange(0,nmems), nino34p['time']],
                                    dims=['Member','time'], name='nino34p')
            
            nino34_fut=xr.DataArray(np.zeros([nmems, nino34f.time.size]),
                                    coords=[np.arange(0,nmems), nino34f['time']],
                                    dims=['Member','time'], name='nino34f')

            prcal_past=xr.DataArray(np.zeros([nmems, nino34p.time.size]),
                                    coords=[np.arange(0,nmems), nino34p['time']],
                                    dims=['Member','time'], name='calpp')

            prcal_fut=xr.DataArray(np.zeros([nmems, nino34f.time.size]),
                                    coords=[np.arange(0,nmems), nino34f['time']],
                                    dims=['Member','time'], name='calpf')

        prpmskd = xr.DataArray(np.array(prdjfpinterp)*np.array(mask), coords=prdjfpinterp.coords)
        prcalp = avg.cosweightlonlat(prpmskd, 0, 360, -90, 90)
        
        prfmskd = xr.DataArray(np.array(prdjffinterp)*np.array(mask), coords=prdjffinterp.coords)
        prcalf = avg.cosweightlonlat(prfmskd, 0, 360, -90, 90)
            
        nino34_past[imem,:]=np.array(nino34p[:])
        nino34_fut[imem,:]=np.array(nino34f[:])
        prcal_past[imem,:]=np.array(prcalp[:])
        prcal_fut[imem,:]=np.array(prcalf[:])   
    
    nino34pdetrend = xr.DataArray(signal.detrend(nino34_past, axis=1), coords=nino34_past.coords)
    calppdetrend = xr.DataArray(signal.detrend(prcal_past, axis=1), coords=prcal_past.coords)
    
    cor_nino34_calp_past = xr.DataArray(np.zeros([nmems]), coords=[np.arange(0,nmems)], dims=['Member'], name='cor_nino34_calp')
    for imem in np.arange(0,nmems,1):
        cor_nino34_calp_past[imem] = xr.corr(calppdetrend[imem,:],nino34pdetrend[imem,:])
        
    cor_nino34_calp_past.to_netcdf(pathout+'calpdata_'+modnames[imod]+'_past.nc')
    nino34_past.rename("nino34_past")
    nino34_fut.rename("nino34_fut")
    prcal_past.rename("calp_past")
    prcal_fut.rename("calp_fut")
    nino34_past.to_netcdf(pathout+'calpdata_'+modnames[imod]+'_past.nc', mode='a')
    nino34_fut.to_netcdf(pathout+'calpdata_'+modnames[imod]+'_future.nc')
    prcal_past.to_netcdf(pathout+'calpdata_'+modnames[imod]+'_past.nc', mode='a')
    prcal_fut.to_netcdf(pathout+'calpdata_'+modnames[imod]+'_future.nc', mode='a')

CanESM2
pr_Amon_CanESM2_historical_rcp85_r10i1p1_195001-210012.nc
ts_Amon_CanESM2_historical_rcp85_r10i1p1_195001-210012.nc
masking California
pr_Amon_CanESM2_historical_rcp85_r11i1p1_195001-210012.nc
ts_Amon_CanESM2_historical_rcp85_r11i1p1_195001-210012.nc
pr_Amon_CanESM2_historical_rcp85_r12i1p1_195001-210012.nc
ts_Amon_CanESM2_historical_rcp85_r12i1p1_195001-210012.nc
pr_Amon_CanESM2_historical_rcp85_r13i1p1_195001-210012.nc
ts_Amon_CanESM2_historical_rcp85_r13i1p1_195001-210012.nc
pr_Amon_CanESM2_historical_rcp85_r14i1p1_195001-210012.nc
ts_Amon_CanESM2_historical_rcp85_r14i1p1_195001-210012.nc
pr_Amon_CanESM2_historical_rcp85_r15i1p1_195001-210012.nc
ts_Amon_CanESM2_historical_rcp85_r15i1p1_195001-210012.nc
pr_Amon_CanESM2_historical_rcp85_r16i1p1_195001-210012.nc
ts_Amon_CanESM2_historical_rcp85_r16i1p1_195001-210012.nc
pr_Amon_CanESM2_historical_rcp85_r17i1p1_195001-210012.nc
ts_Amon_CanESM2_historical_rcp85_r17i1p1_195001-210012.nc
pr_Amon_CanESM2_historical_rcp85_r18i1p1_1950

In [10]:
print(nino34p)

<xarray.DataArray (time: 66)>
dask.array<truediv, shape=(66,), dtype=float64, chunksize=(66,), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) object 1951-01-16 12:00:00 ... 2016-01-16 12:00:00


In [5]:
print(nmonthsf)

1117


In [6]:
print,1117/12.


(<function print>, 93.08333333333333)